SQL RAG AGENT

In [2]:
import os
from dotenv import load_dotenv 

load_dotenv()

GROQ_API=os.environ.get("GROQ_API_KEY")
DB_UNAME=os.environ.get("DB_UNAME")
DB_PASS=os.environ.get("DB_PASS")

In [3]:
from langchain_text_splitters import CharacterTextSplitter
from glob import glob

text_splitter = CharacterTextSplitter(
    separator="###",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

texts = []
for file_path in glob("docs/*.txt"): # loading all text files in docs folder
    with open(file_path, "r") as f:
        doc = f.read()
    
    file_chunks = text_splitter.create_documents(
        [doc], 
        metadatas=[{"source": file_path}]
    )
    texts.extend(file_chunks)

if texts:
    print(texts[0])

/home/ajay/miniconda3/envs/agents/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Created a chunk of size 2410, which is longer than the specified 1000
Created a chunk of size 1075, which is longer than the specified 1000
Created a chunk of size 1835, which is longer than the specified 1000


page_content='DOC_TYPE=INFORMATION
VERSION=2.0
DOMAIN=BANKING
SOURCE=internal' metadata={'source': 'docs/Info.txt'}


In [4]:
texts[-1].page_content

"Common Analytical Intent Mapping\n\n*   **Customer's total money**: `SUM(accounts.balance) + SUM(loans.loan_amount)` for a given `customer_id`.\n*   **Branch performance**: Aggregate `accounts.balance` or `loans.loan_amount` by `branches.branch_name`.\n*   **Loan delinquency**: Find `repayments` where `paid_date IS NULL` and `due_date < CURRENT_DATE`.\n*   **Customers needing KYC**: `SELECT * FROM customers WHERE kyc_status = 'PENDING';`\n*   **Inactive Customers**: Customers with no accounts or loans.\n*   **Branch with most customers**: `COUNT(DISTINCT customer_id)` grouped by `branch_name`."

In [5]:
texts

[Document(metadata={'source': 'docs/Info.txt'}, page_content='DOC_TYPE=INFORMATION\nVERSION=2.0\nDOMAIN=BANKING\nSOURCE=internal'),
 Document(metadata={'source': 'docs/Info.txt'}, page_content='Table Details & Relationships:\n\n**Branches Table (`branches`)**\n*   `branch_id` (UUID, Primary Key): Unique identifier for a branch.\n*   `branch_name` (VARCHAR): Name of the branch (e.g., "Main Street Branch").\n*   `location` (VARCHAR): City or address of the branch.\n\n**Customers Table (`customers`)**\n*   `customer_id` (UUID, Primary Key): Unique identifier for a customer.\n*   `first_name`, `last_name` (VARCHAR): Customer\'s name.\n*   `email` (VARCHAR): Customer\'s email address.\n*   `phone_number` (VARCHAR): Customer\'s phone number.\n*   `kyc_status` (VARCHAR): Know Your Customer status (\'COMPLETED\', \'PENDING\').\n\n**Accounts Table (`accounts`)**\n*   `account_id` (UUID, Primary Key): Unique identifier for an account.\n*   `account_number` (VARCHAR, Unique): Public-facing accoun

In [6]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

schema_store = Chroma.from_documents(
    texts,
    embedding=embeddings,
    collection_name="schema_rag"
)

schema_retriever = schema_store.as_retriever(k=4)

In [7]:
schema_retriever.invoke("currency")

[Document(metadata={'source': 'docs/Info.txt'}, page_content='Core Querying Rules\n\n**1. Soft Deletes:**\n*   **ALL tables have a `deleted_at` (TIMESTAMPTZ, Nullable) column.**\n*   A record is considered active and usable if `deleted_at IS NULL`.\n*   **IMPORTANT**: Unless specifically asked about historical or deleted data, YOU MUST add `WHERE <table_name>.deleted_at IS NULL` for every table in your query. For joins, apply this to all joined tables.\n*   **Example**: `SELECT c.first_name, a.balance FROM customers c JOIN accounts a ON c.customer_id = a.customer_id WHERE c.deleted_at IS NULL AND a.deleted_at IS NULL;`\n\n**2. Currency:**\n*   All monetary values are in **Indian Rupees (INR)**.\n*   When presenting a single monetary value in a sentence, use "Rs." as the prefix.\n*   **Example**: "The total balance of all accounts is Rs. 1,234,567.89."\n\n**3. UUIDs and Timestamps:**\n*   Primary keys are UUIDs (`_id` suffix). Do not infer any meaning from them.\n*   All tables have `cr

In [ ]:
from langchain_core.tools import tool
from sqlalchemy import create_engine, text
from typing import List, Tuple

username = DB_UNAME
password = DB_PASS
db_ip = "localhost"
db_port = "5432"
database = "postgres"
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{db_ip}:{db_port}/{database}")


def validate_sql(query: str):
    q = query.lower().strip()

    if not q.startswith("select"):
        raise ValueError("Only SELECT queries are allowed")

    forbidden = ["insert", "update", "delete", "drop", "alter", ";"]
    # pass_kw = ['deleted_at'] # all queries will have deleted_at 
    
    for kw in forbidden:
        if kw in q:
            raise ValueError(f"Forbidden SQL keyword detected: {kw}")

@tool
def query_vecdb(question: str) -> str:
    """
    Retrieve relevant database schema based on the user question.
    Always call this before generating SQL.
    """
    docs = schema_retriever.invoke(question)
    return "\n\n".join(d.page_content for d in docs)


@tool
def run_sql_query(query: str) -> str:
    """Execute a validated read-only SQL query."""
    
    print("\n--- SQL GENERATED ---")
    print(query)

    # validate_sql(query) # need to solve deleted_at getting caught in forbidden words list

    with engine.connect() as conn:
        result = conn.execute(text(query))
        rows = result.fetchmany(5)
        return str(rows)


@tool
def get_table_schema(table_name: str) -> List[Tuple]:
    """Return column schema information for a table.
    Argument: table_name, example - customers
    """
    query = """
        SELECT column_name, data_type, is_nullable
        FROM information_schema.columns
        WHERE table_name = :table_name
        ORDER BY ordinal_position
    """
    with engine.connect() as conn:
        result = conn.execute(text(query), {"table_name": table_name})
        return result.fetchall()

# need more time and work to implement this
# @tool
# def request_clarification(reason: str) -> str:
#     """Ask the user for clarification before generating SQL."""
#     return f"Clarification needed: {reason}"


@tool
def list_table():
    """Return names of table present in the database."""
    query = """
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema='public';
    """
    with engine.connect() as conn:
        result = conn.execute(text(query))
        return str(result.fetchall())


# @tool
# def verify_column(table: str, column: str) -> bool:
#     """Verify a column exists in a given table."""
#     # this query is not running 
#     query = """ 
#         SELECT 1
#         FROM information_schema.columns
#         WHERE table_name = :table AND column_name = :column
#     """
#     with engine.connect() as conn:
#         result = conn.execute(text(query), {"table": table, "column": column})
#         return result.first() is not None


In [17]:
def test_query_vecdb(question: str) -> str:
    """
    Retrieve relevant database schema based on the user question.
    Always call this before generating SQL.
    """
    docs = schema_retriever.invoke(question)
    return "\n\n".join(d.page_content for d in docs)


def test_run_sql_query(query: str) -> str:
    """Execute a validated read-only SQL query."""
    
    print("\n--- SQL GENERATED ---")
    print(query)

    # validate_sql(query) # need to solve deleted_at getting caught in forbidden words list

    with engine.connect() as conn:
        result = conn.execute(text(query))
        rows = result.fetchmany(5)
        return str(rows)


def test_get_table_schema(table_name: str) -> List[Tuple]:
    """Return column schema information for a table.
    Argument: table_name, example - customers
    """
    query = """
        SELECT column_name, data_type, is_nullable
        FROM information_schema.columns
        WHERE table_name = :table_name
        ORDER BY ordinal_position
    """
    with engine.connect() as conn:
        result = conn.execute(text(query), {"table_name": table_name})
        return result.fetchall()


def test_request_clarification(reason: str) -> str:
    """Ask the user for clarification before generating SQL."""
    return f"Clarification needed: {reason}"


def test_list_table():
    """Return names of table present in the database."""
    query = """
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema='public';
    """
    with engine.connect() as conn:
        result = conn.execute(text(query))
        return str(result.fetchall())
    
print(test_query_vecdb("currency"))
print("-x-x-x-x-x-x-" * 7, "\n\n")
print(test_run_sql_query("select * from loans order by principal desc;"))    
print("-x-x-x-x-x-x-" * 7, "\n\n")
print(test_get_table_schema("customers"))    
print("-x-x-x-x-x-x-" * 7, "\n\n")
print(test_request_clarification("should i execute delete table query?"))
print("-x-x-x-x-x-x-" * 7, "\n\n")
print(test_list_table())

Core Querying Rules

**1. Soft Deletes:**
*   **ALL tables have a `deleted_at` (TIMESTAMPTZ, Nullable) column.**
*   A record is considered active and usable if `deleted_at IS NULL`.
*   **IMPORTANT**: Unless specifically asked about historical or deleted data, YOU MUST add `WHERE <table_name>.deleted_at IS NULL` for every table in your query. For joins, apply this to all joined tables.
*   **Example**: `SELECT c.first_name, a.balance FROM customers c JOIN accounts a ON c.customer_id = a.customer_id WHERE c.deleted_at IS NULL AND a.deleted_at IS NULL;`

**2. Currency:**
*   All monetary values are in **Indian Rupees (INR)**.
*   When presenting a single monetary value in a sentence, use "Rs." as the prefix.
*   **Example**: "The total balance of all accounts is Rs. 1,234,567.89."

**3. UUIDs and Timestamps:**
*   Primary keys are UUIDs (`_id` suffix). Do not infer any meaning from them.
*   All tables have `created_at` and `updated_at` for auditing. These are for internal tracking and 

In [29]:
from langchain_groq import ChatGroq

# "moonshotai/kimi-k2-instruct-0905"
llm = ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

SYSTEM_PROMPT = """
You are a PostgreSQL database assistant.

You MUST follow this exact sequence:
1. Call schema checking tool
2. Read schema carefully
3. Decide required tables and columns
4. Generate a SELECT-only SQL query
5. Execute it using run_sql_query
6. Answer in plain English

Before writing SQL:
- List tables you will use
- List columns you will use
- Verify each exists in schema

Rules:
- Never generate SQL before schema is retrieved
- Never guess table or column names
- Never modify data
- If schema is insufficient, ask for clarification
- You're allowed to execute only data retrieval queries
- Do not execute any data modification or destrunction queries
- If user asks to modify or delete data, refuse and stop execution and explain you can only run read-only queries

After writing the SQL:
- Run it with sql query runner tool
- Only return the result if the query tool gives an output
- If there's an error rework on it
"""

In [30]:
from langchain.agents import create_agent

agent = create_agent(
    llm,
    tools=[
        query_vecdb,
        get_table_schema,
        list_table,
        run_sql_query,
        # request_clarification,
        # verify_column
    ],
    system_prompt=SYSTEM_PROMPT,
)

In [20]:
question = "I need info of loans which were disbursed between jan 2023 and june 2023"

In [27]:
def streamer_agent(question):
    for step in agent.stream(
        {"messages": [{"role": "user", "content": question}]},
        stream_mode="values",
    ):
        yield step["messages"][-1].pretty_print()

In [31]:
[msg for msg in streamer_agent("delete the column tenure_months from loans table")]

================================ Human Message =================================

delete the column tenure_months from loans table
================================== Ai Message ==================================

I can't perform that action. I'm a PostgreSQL database assistant and I'm limited to running read-only queries. I can only provide information about the database schema and execute SELECT queries.

If you'd like to know the schema of the loans table, I can try to retrieve it for you:
Tool Calls:
  get_table_schema (bde68nfgm)
 Call ID: bde68nfgm
  Args:
    table_name: loans
================================= Tool Message =================================
Name: get_table_schema

[('loan_id', 'uuid', 'NO'), ('customer_id', 'uuid', 'NO'), ('branch_id', 'uuid', 'NO'), ('loan_type', 'text', 'YES'), ('principal', 'numeric', 'YES'), ('interest_rate', 'numeric', 'YES'), ('tenure_months', 'integer', 'YES'), ('disbursed_at', 'date', 'YES'), ('status', 'text', 'YES'), ('created_at', 'time

[None, None, None, None]

In [22]:
[msg for msg in streamer_agent(question)]

================================ Human Message =================================

I need info of loans which were disbursed between jan 2023 and june 2023
================================== Ai Message ==================================

To provide the information about loans disbursed between January 2023 and June 2023, I first need to understand the schema of the database, specifically the tables and columns related to loans.

Let's start by retrieving the relevant database schema based on the user question.
Tool Calls:
  query_vecdb (g0vxmv8hp)
 Call ID: g0vxmv8hp
  Args:
    question: loans disbursed between jan 2023 and june 2023
================================= Tool Message =================================
Name: query_vecdb

Common Analytical Intent Mapping

*   **Customer's total money**: `SUM(accounts.balance) + SUM(loans.loan_amount)` for a given `customer_id`.
*   **Branch performance**: Aggregate `accounts.balance` or `loans.loan_amount` by `branches.branch_name`.
*   **Loa

[None, None, None, None, None, None, None, None, None, None]

In [23]:
[msg for msg in streamer_agent("just give me the names of all tables")]

================================ Human Message =================================

just give me the names of all tables
================================== Ai Message ==================================
Tool Calls:
  list_table (8vwxfrfaq)
 Call ID: 8vwxfrfaq
  Args:
================================= Tool Message =================================
Name: list_table

[('branches',), ('customers',), ('accounts',), ('loans',), ('repayments',)]
================================== Ai Message ==================================

The tables present in the database are: 
- branches
- customers
- accounts
- loans
- repayments


[None, None, None, None]

In [24]:
[msg for msg in streamer_agent("How financially exposed is the bank to each customer?")]

================================ Human Message =================================

How financially exposed is the bank to each customer?
================================== Ai Message ==================================

To assess the financial exposure of the bank to each customer, we need to understand the relevant tables and columns in the database. Let's start by querying the database schema.
Tool Calls:
  query_vecdb (h1adgq98e)
 Call ID: h1adgq98e
  Args:
    question: bank customer financial exposure
================================= Tool Message =================================
Name: query_vecdb

Common Analytical Intent Mapping

*   **Customer's total money**: `SUM(accounts.balance) + SUM(loans.loan_amount)` for a given `customer_id`.
*   **Branch performance**: Aggregate `accounts.balance` or `loans.loan_amount` by `branches.branch_name`.
*   **Loan delinquency**: Find `repayments` where `paid_date IS NULL` and `due_date < CURRENT_DATE`.
*   **Customers needing KYC**: `SELECT *

ProgrammingError: (psycopg2.errors.UndefinedColumn) column c.first_name does not exist
LINE 1: SELECT     c.customer_id,     c.first_name,     c.last_name,...
                                      ^

[SQL: SELECT     c.customer_id,     c.first_name,     c.last_name,     COALESCE(SUM(a.balance), 0) AS total_account_balance,     COALESCE(SUM(l.loan_amount), 0) AS total_loan_amount,     COALESCE(SUM(a.balance), 0) + COALESCE(SUM(l.loan_amount), 0) AS total_financial_exposure FROM     customers c LEFT JOIN     accounts a ON c.customer_id = a.customer_id AND a.deleted_at IS NULL LEFT JOIN     loans l ON c.customer_id = l.customer_id AND l.deleted_at IS NULL GROUP BY     c.customer_id, c.first_name, c.last_name ORDER BY     total_financial_exposure DESC;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
[msg for msg in streamer_agent("whats the sum of outstanding loan amount of all customer?")]

================================ Human Message =================================

whats the sum of outstanding loan amount of all customer?
================================== Ai Message ==================================
Tool Calls:
  query_vecdb (mgrvkq9n5)
 Call ID: mgrvkq9n5
  Args:
    question: whats the sum of outstanding loan amount of all customer?
================================= Tool Message =================================
Name: query_vecdb

Common Analytical Intent Mapping

Customer financial exposure → accounts + loans
Branch performance → aggregate accounts and loans by branch
Loan health → loan status + unpaid repayments
Regulatory checks → customers with kyc_status = 'PENDING'
Delinquency analysis → repayments where paid_date IS NULL AND due_date < CURRENT_DATE

DOC_TYPE=INFORMATION
VERSION=1.0
DOMAIN=BANKING
SOURCE=internal

Soft deleted using deleted_at
created_at and updated_at are system-managed

### Table Details:
Customers Table: represents individual bank custo

[None, None, None, None, None, None, None, None, None, None]